<a href="https://colab.research.google.com/github/maro96ost/BINA_Shawarma/blob/main/AutoScout_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###BINA: Data Cleaning

In the first part, we scraped all the data from AutoScout.ch. Now it's time to clean the data and get rid of mistaken or incomplete datasets.

First we start with importing the different libraries


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

##Importing files
* Import the needed files from Google Drive

First we need the csv file with all the data from AutoScout. Second we import another csv, which consists of brand and model names. This one will be used to match the cars with their brand / model.

In [ ]:
#import csv files with all AutoScout Data
file_id = '12sfOuYpWJ6sVb03sY6mPg2ESpSeuCXbp'
direct_link = f'https://drive.google.com/uc?id={file_id}'

df = pd.read_csv(direct_link)
df.head()

,Car,Price,Date,Km,Transmission,Fuel,PS,Consumption,Location
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,CHF 44'450.–,02.2020,45'486 km,Automat,Benzin,224 PS (165 kW),8.9 l/100 km,8408 Winterthur (Wülflingen)
1,AUDI RS e-tron GT quattro,CHF 155'990.–,Neues Fahrzeug,9 km,Automatikgetriebe,Elektro,599 PS (440 kW),469 km,4147 Aesch
2,BMW 120d xDrive Sport Line,CHF 36'900.–,05.2023,28'100 km,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
3,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,CHF 18'799.–,05.2012,68'200 km,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
4,SEAT Leon ST 2.0 TSI Cupra 290 DSG,CHF 21'900.–,03.2016,86'000 km,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal


In [ ]:
#import csv file with all Brand/Model Names
file_id2 = '1B4q2v-wbFcewrc8pR2LNv86quLORUrXE'
direct_link2 = f'https://drive.google.com/uc?id={file_id2}'

dfb = pd.read_csv(direct_link2)

In [ ]:
#drop the column "Unnamed: 0"
dfb.drop("Unnamed: 0", axis=1, inplace=True)

##Data Cleaning
From now on we start cleaning our data. This will be completed step by step.

* Clean column Date --> vaild date format
* Create new columns for Neuwagen, Vorführmodell, Occasion
* Set date for Neuwagen & Vorführmodell equal 01.2025
* Drop cars without date

In [ ]:
# Regular expression for date format MM.YYYY
date_pattern = r'^\d{2}\.\d{4}$'

# Filter for values that do NOT match the date format
non_date_values = df[~df['Date'].str.match(date_pattern)]['Date'].unique()
print("Unique non-date values in the 'Date' column:", non_date_values)

#Create new Columns Vorführmodell, Neuwagen, Occasion


Unique non-date values in the 'Date' column: ['Neues Fahrzeug' 'Vorführmodell' '2024' '2014' '2015' '2009' '2006'
 '2003' '2008' '2021' '2012' '2013']


###Creating new columns

If value for column date equal "Neues Fahrzeug" or "Vorführmodell" set the respective column = 1

If column date equals a valid date, set column occasion equal 1



In [ ]:
# Create new columns with default 0
df['Neuwagen'] = 0
df['Vorführmodell'] = 0
df['Occasion'] = 0

# Set conditions
df.loc[df['Date'] == 'Neues Fahrzeug', 'Neuwagen'] = 1
df.loc[df['Date'] == 'Vorführmodell', 'Vorführmodell'] = 1
# Assuming "Occasion" is true when neither "Neuwagen" nor "Vorführmodell"
df['Occasion'] = ((df['Neuwagen'] == 0) & (df['Vorführmodell'] == 0)).astype(int) #

# Reorder columns to insert the new columns after 'car'
col_order = df.columns.tolist()
# Move 'Neuwagen', 'Vorführmodell', 'Occasion' right after 'car'
new_cols = ['Neuwagen', 'Vorführmodell', 'Occasion']
for col in reversed(new_cols):  # Reverse to keep the order correct after insert
    col_order.insert(2, col_order.pop(col_order.index(col)))

df = df[col_order]




In [ ]:
df.head()

,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,CHF 44'450.–,0,0,1,02.2020,45'486 km,Automat,Benzin,224 PS (165 kW),8.9 l/100 km,8408 Winterthur (Wülflingen)
1,AUDI RS e-tron GT quattro,CHF 155'990.–,1,0,0,Neues Fahrzeug,9 km,Automatikgetriebe,Elektro,599 PS (440 kW),469 km,4147 Aesch
2,BMW 120d xDrive Sport Line,CHF 36'900.–,0,0,1,05.2023,28'100 km,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
3,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,CHF 18'799.–,0,0,1,05.2012,68'200 km,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
4,SEAT Leon ST 2.0 TSI Cupra 290 DSG,CHF 21'900.–,0,0,1,03.2016,86'000 km,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal


### Cleaning column Date
To ensure that in the column date we only have valid date values, we need to tranform the values "Neues Fahrzeug" and "Vorführmodell" in column date to correct date values.

So all cars with value "Neues Fahrzeug" or "Vorführmodell" in the column date, are updated to '01.2025'.

In addition, all rows are doped, where date is NaN



In [ ]:
#correct neues Fahrzeug / Vorführmodell
df['Date'] = df['Date'].str.replace('Neues Fahrzeug', '01.2025', regex=False)
df['Date'] = df['Date'].str.replace('Vorführmodell', '01.2025', regex=False)

#correct date format
df['Date'] = df['Date'].apply(lambda x: '01.' + x if len(x) == 4 and x.isdigit() else x) # Die Lambda-Funktion überprüft, ob die Länge des Werts 4 beträgt und ob er nur aus Ziffern besteht. Wenn ja, wird '01.' vor den Wert gesetzt, um ein korrektes Datumsformat sicherzustellen. Andernfalls bleibt der Wert unverändert.

#Change Datatype
df['Date'] = pd.to_datetime(df['Date'], format='%m.%Y', errors='coerce') # format='%m.%Y': Dieser Parameter gibt das erwartete Format der Datumszeichenfolge in der Spalte 'Date' an. Hier bedeutet '%m.%Y', dass das Datum als Monat und Jahr erwartet wird, wobei der Monat durch eine Zahl und das Jahr durch vier Ziffern dargestellt wird, getrennt durch einen Punkt (z.B. '01.2022' für Januar 2022).
# errors='coerce': Dieser Parameter gibt an, wie mit Fehlern bei der Umwandlung umgegangen werden soll. 'coerce' bedeutet, dass fehlerhafte Eingaben in NaT (Not a Time) umgewandelt werden, was praktisch ist, wenn das Datum nicht im erwarteten Format vorliegt oder ungültige Daten vorhanden sind.

In [ ]:
#Drop car with Date is NaT
df_nat = df[pd.isna(df['Date'])]
df.drop(index=df_nat.index)


,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,CHF 44'450.–,0,0,1,2020-02-01,45'486 km,Automat,Benzin,224 PS (165 kW),8.9 l/100 km,8408 Winterthur (Wülflingen)
1,AUDI RS e-tron GT quattro,CHF 155'990.–,1,0,0,2025-01-01,9 km,Automatikgetriebe,Elektro,599 PS (440 kW),469 km,4147 Aesch
2,BMW 120d xDrive Sport Line,CHF 36'900.–,0,0,1,2023-05-01,28'100 km,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
3,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,CHF 18'799.–,0,0,1,2012-05-01,68'200 km,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
4,SEAT Leon ST 2.0 TSI Cupra 290 DSG,CHF 21'900.–,0,0,1,2016-03-01,86'000 km,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal
...,...,...,...,...,...,...,...,...,...,...,...,...
158555,ALFA ROMEO 147 1.9 16V JTD Q2,CHF 3'890.–,0,0,1,2009-11-01,226'220 km,Schaltgetriebe manuell,Diesel,150 PS (110 kW),5.9 l/100 km,9113 Degersheim
158556,MERCEDES-BENZ SL 500 Automatic,CHF 17'900.–,0,0,1,2002-07-01,75'000 km,Automatikgetriebe,Benzin,306 PS (225 kW),12.1 l/100 km,8051 Zürich/ Schwamendingen
158557,SUBARU Forester 2.0D Luxury Lineatronic,CHF 14'500.–,0,0,1,2015-08-01,106'700 km,Stufenlos,Diesel,147 PS (108 kW),6.1 l/100 km,8253 Diessenhofen
158558,PEUGEOT 407 SW 2.0 HDI SR Selection,CHF 1'700.–,0,0,1,2006-10-01,152'347 km,Automat,Diesel,136 PS (100 kW),7.2 l/100 km,1713 St. Antoni


Show all datatypes of dataframe df

In [ ]:
df.dtypes
#158560 rows × 13 columns

Car                      object
Price                    object
Neuwagen                  int64
Vorführmodell             int64
Occasion                  int64
Date             datetime64[ns]
Km                       object
Transmission             object
Fuel                     object
PS                       object
Consumption              object
Location                 object
dtype: object

Show all duplicates where car is not Neuwagen and is not Vorführmodell --> Occasion cars

In [ ]:
# Filter out the rows where 'Neuwagen' or 'Vorführmodell' equal 1
df_filtered = df[(df['Neuwagen'] != 1) & (df['Vorführmodell'] != 1)]

# Find duplicates in the filtered DataFrame
duplicates = df_filtered[df_filtered.duplicated(keep=False)]

# If you want to see these duplicates in the context of the original DataFrame:
duplicates_in_original_df = df.loc[duplicates.index]
duplicates_in_original_df


,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location
146,SKODA OCTAVIA 2.0 TSI RS DSG,CHF 39'980.–,0,0,1,2023-05-01,50 km,Halbautomatisches Getriebe,Benzin,245 PS,7.3 l/100 km,9542 Münchwilen
209,SKODA Octavia Combi 2.0 TDI RS 4x4 DSG,CHF 36'800.–,0,0,1,2022-05-01,31'200 km,Halbautomatisches Getriebe,Diesel,200 PS (147 kW),6.2 l/100 km,9320 Arbon
549,FIAT TIPO 1.5 Hybrid DCT Station Wagon,CHF 23'889.–,0,0,1,2023-10-01,20 km,Automatikgetriebe,Voll-Hybrid Benzin/Elektro,130 PS (96 kW),5.4 l/100 km,5610 Wohlen (AG)
824,VW E-UP ***TOP AUSSTATTUNG***,CHF 23'900.–,0,0,1,2023-11-01,20 km,Automat,Elektro,82 PS (60 kW),225 km,8404 Winterthur
2300,TOYOTA Aygo X 1.0 VVT-i Trend,CHF 18'900.–,0,0,1,2023-11-01,10 km,Schaltgetriebe manuell,Benzin,72 PS (53 kW),-,1196 Gland
...,...,...,...,...,...,...,...,...,...,...,...,...
158260,AUDI A5 Cabriolet 2.0 TFSI quattro S-tronic,CHF 14'499.–,0,0,1,2010-12-01,124'000 km,Halbautomatisches Getriebe,Benzin,211 PS (155 kW),7.7 l/100 km,3297 Leuzigen
158499,MASERATI Quattroporte 4.2 V8 DuoSelect,CHF 24'500.–,0,0,1,2004-12-01,77'000 km,Halbautomatisches Getriebe,Benzin,400 PS (295 kW),18.9 l/100 km,8505 Pfyn
158500,MASERATI Quattroporte 4.2 V8 DuoSelect,CHF 24'500.–,0,0,1,2004-12-01,77'000 km,Halbautomatisches Getriebe,Benzin,400 PS (295 kW),18.9 l/100 km,8505 Pfyn
158539,VW Golf Variant 1.4 TSI Comfortline DSG,CHF 6'900.–,0,0,1,2013-12-01,204'000 km,Halbautomatisches Getriebe,Benzin,122 PS (90 kW),5.1 l/100 km,1868 Collombey


##Drop dupicates
Drop all duplicates where car is Occasion

In [ ]:
# Assuming 'df' is your DataFrame

# Separate the DataFrame into two parts
df_no_special_rows = df[(df['Neuwagen'] != 1) & (df['Vorführmodell'] != 1)]
df_special_rows = df[(df['Neuwagen'] == 1) | (df['Vorführmodell'] == 1)]

# Drop duplicates from the part of the DataFrame without special rows
df_no_special_rows = df_no_special_rows.drop_duplicates(keep=False)

# Append the special rows back to the DataFrame without duplicates
df_cleaned = df_no_special_rows.append(df_special_rows, ignore_index=True)

df_cleaned


<ipython-input-13-aad3b1b600d4>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cleaned = df_no_special_rows.append(df_special_rows, ignore_index=True)


,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,CHF 44'450.–,0,0,1,2020-02-01,45'486 km,Automat,Benzin,224 PS (165 kW),8.9 l/100 km,8408 Winterthur (Wülflingen)
1,BMW 120d xDrive Sport Line,CHF 36'900.–,0,0,1,2023-05-01,28'100 km,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,CHF 18'799.–,0,0,1,2012-05-01,68'200 km,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,CHF 21'900.–,0,0,1,2016-03-01,86'000 km,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal
4,VW Touareg 3.0 TDI R Line Tiptronic,CHF 46'900.–,0,0,1,2018-06-01,99'500 km,Automat,Diesel,286 PS (210 kW),8.8 l/100 km,8153 Rümlang ZH
...,...,...,...,...,...,...,...,...,...,...,...,...
156177,SKODA Fabia 1.5 TSI Style DSG,CHF 27'990.–,1,0,0,2025-01-01,25 km,Halbautomatisches Getriebe,Benzin,150 PS (110 kW),-,2052 Fontainemelon
156178,JAC E-JS1,CHF 17'689.–,1,0,0,2025-01-01,10 km,-,Elektro,61 PS (45 kW),330 km,9427 Wolfhalden
156179,ISUZU,CHF 48'200.–,1,0,0,2025-01-01,100 km,Schaltgetriebe manuell,-,-,-,9033 Untereggen
156180,"JAC JAC e-S4 65,7 kWh",CHF 29'989.–,1,0,0,2025-01-01,10 km,-,Elektro,193 PS (142 kW),410 km,9427 Wolfhalden


After cleaning duplicate Occasion cars, we want to check how many duplicates are in the categories "Neuwagen" and "Vorführmodell".

Total: 2906 duplicates

This is realistic, as a garage could have multiple car's from the same model listed to sell.



In [ ]:
# check if df_cleaned has duplicates - Vorführwagen und Neuwagen werent cleaned yet!
# Check for duplicates in the cleaned DataFrame
duplicates_in_df_cleaned = df_cleaned.duplicated().sum()

# Print the number of duplicates
print(f"Number of duplicate rows in df_cleaned: {duplicates_in_df_cleaned}")

# If you need more information about the duplicates, such as viewing them
if duplicates_in_df_cleaned > 0:
    # Get the duplicate rows
    duplicate_rows = df_cleaned[df_cleaned.duplicated(keep=False)]
    print("Duplicate rows in df_cleaned:")
    print(duplicate_rows)


Number of duplicate rows in df_cleaned: 2906
Duplicate rows in df_cleaned:
                                                      Car         Price  \
120952  VW Tiguan 1.4TSIPHEV R-L DSG 245PS 5 Jahre Wer...  CHF 50'900.–   
120955           SKODA Octavia Combi 2.0 TDI DSG Ambition  CHF 39'990.–   
120977                        SKODA Fabia 1.0 MPI Essence  CHF 18'990.–   
120978                        SKODA Fabia 1.0 MPI Essence  CHF 18'990.–   
120979                        SKODA Fabia 1.0 MPI Essence  CHF 18'990.–   
...                                                   ...           ...   
156032      TOYOTA RAV-4 RAV4 2.5 Plug-In-Hybrid Platinum  CHF 66'200.–   
156034                   TOYOTA Yaris 1.5 VVT-i HSD Trend  CHF 28'890.–   
156039                   TOYOTA Yaris 1.5 VVT-i HSD Trend  CHF 28'890.–   
156045            SUZUKI S-Cross 1.5 Piz Sulai Top Hybrid  CHF 39'780.–   
156100            SUZUKI S-Cross 1.5 Piz Sulai Top Hybrid  CHF 39'780.–   

        Neuwagen  Vorfüh

Drop rows where values are NaN

In [ ]:
df_cleaned[df_cleaned.isna().any(axis=1)]
df_cleaned = df_cleaned.dropna()

In [ ]:
df_cleaned

,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,CHF 44'450.–,0,0,1,2020-02-01,45'486 km,Automat,Benzin,224 PS (165 kW),8.9 l/100 km,8408 Winterthur (Wülflingen)
1,BMW 120d xDrive Sport Line,CHF 36'900.–,0,0,1,2023-05-01,28'100 km,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,CHF 18'799.–,0,0,1,2012-05-01,68'200 km,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,CHF 21'900.–,0,0,1,2016-03-01,86'000 km,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal
4,VW Touareg 3.0 TDI R Line Tiptronic,CHF 46'900.–,0,0,1,2018-06-01,99'500 km,Automat,Diesel,286 PS (210 kW),8.8 l/100 km,8153 Rümlang ZH
...,...,...,...,...,...,...,...,...,...,...,...,...
156177,SKODA Fabia 1.5 TSI Style DSG,CHF 27'990.–,1,0,0,2025-01-01,25 km,Halbautomatisches Getriebe,Benzin,150 PS (110 kW),-,2052 Fontainemelon
156178,JAC E-JS1,CHF 17'689.–,1,0,0,2025-01-01,10 km,-,Elektro,61 PS (45 kW),330 km,9427 Wolfhalden
156179,ISUZU,CHF 48'200.–,1,0,0,2025-01-01,100 km,Schaltgetriebe manuell,-,-,-,9033 Untereggen
156180,"JAC JAC e-S4 65,7 kWh",CHF 29'989.–,1,0,0,2025-01-01,10 km,-,Elektro,193 PS (142 kW),410 km,9427 Wolfhalden


### Cleaning column Price
Removing "CHF" and ".-", so that we only have integer values to proceed with calculations.

In [ ]:
# clean Price Column
df_cleaned['Price'] = df_cleaned['Price'].str.replace('CHF ', '', regex=False).str.replace("'", '', regex=False).str.rstrip('.–')
df_cleaned

<ipython-input-17-07043fa6777f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Price'] = df_cleaned['Price'].str.replace('CHF ', '', regex=False).str.replace("'", '', regex=False).str.rstrip('.–')


,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,44450,0,0,1,2020-02-01,45'486 km,Automat,Benzin,224 PS (165 kW),8.9 l/100 km,8408 Winterthur (Wülflingen)
1,BMW 120d xDrive Sport Line,36900,0,0,1,2023-05-01,28'100 km,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,18799,0,0,1,2012-05-01,68'200 km,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,21900,0,0,1,2016-03-01,86'000 km,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal
4,VW Touareg 3.0 TDI R Line Tiptronic,46900,0,0,1,2018-06-01,99'500 km,Automat,Diesel,286 PS (210 kW),8.8 l/100 km,8153 Rümlang ZH
...,...,...,...,...,...,...,...,...,...,...,...,...
156177,SKODA Fabia 1.5 TSI Style DSG,27990,1,0,0,2025-01-01,25 km,Halbautomatisches Getriebe,Benzin,150 PS (110 kW),-,2052 Fontainemelon
156178,JAC E-JS1,17689,1,0,0,2025-01-01,10 km,-,Elektro,61 PS (45 kW),330 km,9427 Wolfhalden
156179,ISUZU,48200,1,0,0,2025-01-01,100 km,Schaltgetriebe manuell,-,-,-,9033 Untereggen
156180,"JAC JAC e-S4 65,7 kWh",29989,1,0,0,2025-01-01,10 km,-,Elektro,193 PS (142 kW),410 km,9427 Wolfhalden


### Cleaning KM column
Same as before...

In [ ]:
# Clean Km column
df_cleaned['Km'] = df_cleaned['Km'].str.replace("'", "").str.replace(" km", "", regex=False)
df_cleaned

<ipython-input-18-287186ef0489>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Km'] = df_cleaned['Km'].str.replace("'", "").str.replace(" km", "", regex=False)


,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,44450,0,0,1,2020-02-01,45486,Automat,Benzin,224 PS (165 kW),8.9 l/100 km,8408 Winterthur (Wülflingen)
1,BMW 120d xDrive Sport Line,36900,0,0,1,2023-05-01,28100,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,18799,0,0,1,2012-05-01,68200,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,21900,0,0,1,2016-03-01,86000,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal
4,VW Touareg 3.0 TDI R Line Tiptronic,46900,0,0,1,2018-06-01,99500,Automat,Diesel,286 PS (210 kW),8.8 l/100 km,8153 Rümlang ZH
...,...,...,...,...,...,...,...,...,...,...,...,...
156177,SKODA Fabia 1.5 TSI Style DSG,27990,1,0,0,2025-01-01,25,Halbautomatisches Getriebe,Benzin,150 PS (110 kW),-,2052 Fontainemelon
156178,JAC E-JS1,17689,1,0,0,2025-01-01,10,-,Elektro,61 PS (45 kW),330 km,9427 Wolfhalden
156179,ISUZU,48200,1,0,0,2025-01-01,100,Schaltgetriebe manuell,-,-,-,9033 Untereggen
156180,"JAC JAC e-S4 65,7 kWh",29989,1,0,0,2025-01-01,10,-,Elektro,193 PS (142 kW),410 km,9427 Wolfhalden


### Cleaning PS Column
in the column PS there are two values. One for PS (Horsepower) and the other for kW (Kilowatt). So that further analysis is easier, this column will be splited in to two columns (PS and kW).

In [ ]:
# clean PS column
df_cleaned[['PS', 'kW']] = df_cleaned['PS'].str.extract(r'(\d+) PS \((\d+) kW\)')
df_cleaned

<ipython-input-19-62459b22a16a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[['PS', 'kW']] = df_cleaned['PS'].str.extract(r'(\d+) PS \((\d+) kW\)')
<ipython-input-19-62459b22a16a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[['PS', 'kW']] = df_cleaned['PS'].str.extract(r'(\d+) PS \((\d+) kW\)')


,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location,kW
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,44450,0,0,1,2020-02-01,45486,Automat,Benzin,224,8.9 l/100 km,8408 Winterthur (Wülflingen),165
1,BMW 120d xDrive Sport Line,36900,0,0,1,2023-05-01,28100,Automat,Diesel,190,5.2 l/100 km,7000 Chur,140
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,18799,0,0,1,2012-05-01,68200,Automat,Diesel,204,6.5 l/100 km,8156 Oberhasli,150
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,21900,0,0,1,2016-03-01,86000,Halbautomatisches Getriebe,Benzin,290,6.6 l/100 km,5417 Untersiggenthal,213
4,VW Touareg 3.0 TDI R Line Tiptronic,46900,0,0,1,2018-06-01,99500,Automat,Diesel,286,8.8 l/100 km,8153 Rümlang ZH,210
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156177,SKODA Fabia 1.5 TSI Style DSG,27990,1,0,0,2025-01-01,25,Halbautomatisches Getriebe,Benzin,150,-,2052 Fontainemelon,110
156178,JAC E-JS1,17689,1,0,0,2025-01-01,10,-,Elektro,61,330 km,9427 Wolfhalden,45
156179,ISUZU,48200,1,0,0,2025-01-01,100,Schaltgetriebe manuell,-,NaN,-,9033 Untereggen,NaN
156180,"JAC JAC e-S4 65,7 kWh",29989,1,0,0,2025-01-01,10,-,Elektro,193,410 km,9427 Wolfhalden,142


### Cleaning column Location
As the column location consists of two values (ZIP + location name), we will also be splitting them.

In [ ]:
df_cleaned[['ZIP', 'Location_Name']] = df_cleaned['Location'].str.extract(r'(\d{4})\s(.*)')

<ipython-input-20-b9eb1ae85c9b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[['ZIP', 'Location_Name']] = df_cleaned['Location'].str.extract(r'(\d{4})\s(.*)')
<ipython-input-20-b9eb1ae85c9b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[['ZIP', 'Location_Name']] = df_cleaned['Location'].str.extract(r'(\d{4})\s(.*)')


In [ ]:
df_cleaned

,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location,kW,ZIP,Location_Name
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,44450,0,0,1,2020-02-01,45486,Automat,Benzin,224,8.9 l/100 km,8408 Winterthur (Wülflingen),165,8408,Winterthur (Wülflingen)
1,BMW 120d xDrive Sport Line,36900,0,0,1,2023-05-01,28100,Automat,Diesel,190,5.2 l/100 km,7000 Chur,140,7000,Chur
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,18799,0,0,1,2012-05-01,68200,Automat,Diesel,204,6.5 l/100 km,8156 Oberhasli,150,8156,Oberhasli
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,21900,0,0,1,2016-03-01,86000,Halbautomatisches Getriebe,Benzin,290,6.6 l/100 km,5417 Untersiggenthal,213,5417,Untersiggenthal
4,VW Touareg 3.0 TDI R Line Tiptronic,46900,0,0,1,2018-06-01,99500,Automat,Diesel,286,8.8 l/100 km,8153 Rümlang ZH,210,8153,Rümlang ZH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156177,SKODA Fabia 1.5 TSI Style DSG,27990,1,0,0,2025-01-01,25,Halbautomatisches Getriebe,Benzin,150,-,2052 Fontainemelon,110,2052,Fontainemelon
156178,JAC E-JS1,17689,1,0,0,2025-01-01,10,-,Elektro,61,330 km,9427 Wolfhalden,45,9427,Wolfhalden
156179,ISUZU,48200,1,0,0,2025-01-01,100,Schaltgetriebe manuell,-,NaN,-,9033 Untereggen,NaN,9033,Untereggen
156180,"JAC JAC e-S4 65,7 kWh",29989,1,0,0,2025-01-01,10,-,Elektro,193,410 km,9427 Wolfhalden,142,9427,Wolfhalden


In [ ]:
# Assuming 'df_cleaned' is your DataFrame and it already has the 'zip_code' column

# Check if all characters in 'zip_code' are numeric
numeric_zip_codes = df_cleaned['ZIP'].str.isnumeric()

# Find out if there are any non-numeric values
any_non_numeric = not numeric_zip_codes.all()

# Print result
print(f"Are all zip code values numeric? {'No, there are non-numeric values.' if any_non_numeric else 'Yes, all values are numeric.'}")

# Optionally, to see the non-numeric zip code values:
if any_non_numeric:
    non_numeric_zip_codes = df_cleaned[~numeric_zip_codes]
    print("Non-numeric zip code values:")
    print(non_numeric_zip_codes['ZIP'])


Are all zip code values numeric? Yes, all values are numeric.


Neue Spaltenordnung definieren

In [ ]:
# Neue Spaltenordnung definieren
new_order = ['Car',
             'Price',
             'Neuwagen',
             'Vorführmodell',
             'Occasion',
             'Date',
             'Km',
             'Transmission',
             'Fuel',
             'PS',
             'kW',
             'Consumption',
             'Location',
             'ZIP',
             'Location_Name']
dfc = df_cleaned[new_order]

In [ ]:
dfc = dfc.drop('Location', axis=1)

### Change datatype where needed
As the columns price, PS, kW and ZIP are numbers, we change them to the datatype numeric.


In [ ]:
dfc['Price'] = pd.to_numeric(dfc['Price'], errors='coerce')
dfc['PS'] = pd.to_numeric(dfc['PS'], errors='coerce')
dfc['kW'] = pd.to_numeric(dfc['kW'], errors='coerce')
dfc['ZIP'] = pd.to_numeric(dfc['ZIP'], errors='coerce')

### Cleaning column consumption
In the actual column consumption we have two different type of values. For gas/diesel cars there is a liter value for 100km. For electric cars the range of the batery is indicated. To work easier with the data, we decided to split this in to two new columns. One column just with the consumption for gas/diesel cars and an othter column for the range of the electric cars.


In [ ]:

# Annahme: dfc ist dein DataFrame

# Kopiere den DataFrame, um Änderungen vorzunehmen
dfc_copy = dfc.copy()

# Extrahiere die Verbrauchswerte ohne Einheiten für "KM (Benzin/Diesel)"
dfc_copy['KM (Benzin/Diesel)'] = dfc_copy['Consumption'].str.extract(r'(\d+\.?\d*)\s*l/100\s*km', expand=False)
dfc_copy['KM (Benzin/Diesel)'] = pd.to_numeric(dfc_copy['KM (Benzin/Diesel)'])

# Extrahiere die Verbrauchswerte ohne Einheiten für "KM (Elektro)"
dfc_copy['KM (Elektro)'] = dfc_copy['Consumption'].str.extract(r'(\d+\.?\d*)\s*km', expand=False)
dfc_copy['KM (Elektro)'] = pd.to_numeric(dfc_copy['KM (Elektro)'])

# Setze "KM (Elektro)" auf NaN, wenn "KM (Benzin/Diesel)" nicht NaN ist
dfc_copy.loc[~dfc_copy['KM (Benzin/Diesel)'].isna(), 'KM (Elektro)'] = pd.NA

# Ersetze alle NaN-Werte in den beiden Spalten durch '-'
dfc_copy['KM (Benzin/Diesel)'].fillna('-', inplace=True)
dfc_copy['KM (Elektro)'].fillna('-', inplace=True)

# Entferne die ursprüngliche "Consumption"-Spalte
dfc_copy.drop(columns=['Consumption'], inplace=True)

dfc = dfc_copy.copy()

# Umbenennen der Spalte "KM (Benzin/Diesel)" in "pro 100/KM (Benzin/Diesel)"
dfc.rename(columns={'KM (Benzin/Diesel)': 'Verbrauch pro 100/KM (Benzin/Diesel)'}, inplace=True)

# Anzeige des aktualisierten DataFrames
dfc_copy = dfc


In [ ]:
dfc

,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,kW,ZIP,Location_Name,Verbrauch pro 100/KM (Benzin/Diesel),KM (Elektro)
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,44450.0,0,0,1,2020-02-01,45486,Automat,Benzin,224.0,165.0,8408,Winterthur (Wülflingen),8.9,-
1,BMW 120d xDrive Sport Line,36900.0,0,0,1,2023-05-01,28100,Automat,Diesel,190.0,140.0,7000,Chur,5.2,-
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,18799.0,0,0,1,2012-05-01,68200,Automat,Diesel,204.0,150.0,8156,Oberhasli,6.5,-
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,21900.0,0,0,1,2016-03-01,86000,Halbautomatisches Getriebe,Benzin,290.0,213.0,5417,Untersiggenthal,6.6,-
4,VW Touareg 3.0 TDI R Line Tiptronic,46900.0,0,0,1,2018-06-01,99500,Automat,Diesel,286.0,210.0,8153,Rümlang ZH,8.8,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156177,SKODA Fabia 1.5 TSI Style DSG,27990.0,1,0,0,2025-01-01,25,Halbautomatisches Getriebe,Benzin,150.0,110.0,2052,Fontainemelon,-,-
156178,JAC E-JS1,17689.0,1,0,0,2025-01-01,10,-,Elektro,61.0,45.0,9427,Wolfhalden,-,330.0
156179,ISUZU,48200.0,1,0,0,2025-01-01,100,Schaltgetriebe manuell,-,NaN,NaN,9033,Untereggen,-,-
156180,"JAC JAC e-S4 65,7 kWh",29989.0,1,0,0,2025-01-01,10,-,Elektro,193.0,142.0,9427,Wolfhalden,-,410.0


In [ ]:
dfc.isna().any()

Car                                     False
Price                                   False
Neuwagen                                False
Vorführmodell                           False
Occasion                                False
Date                                    False
Km                                      False
Transmission                            False
Fuel                                    False
PS                                       True
kW                                       True
ZIP                                     False
Location_Name                           False
Verbrauch pro 100/KM (Benzin/Diesel)    False
KM (Elektro)                            False
dtype: bool

In [ ]:
dfc.dropna(axis=0, inplace=True)

In [ ]:
dfc

,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,kW,ZIP,Location_Name,Verbrauch pro 100/KM (Benzin/Diesel),KM (Elektro)
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,44450.0,0,0,1,2020-02-01,45486,Automat,Benzin,224.0,165.0,8408,Winterthur (Wülflingen),8.9,-
1,BMW 120d xDrive Sport Line,36900.0,0,0,1,2023-05-01,28100,Automat,Diesel,190.0,140.0,7000,Chur,5.2,-
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,18799.0,0,0,1,2012-05-01,68200,Automat,Diesel,204.0,150.0,8156,Oberhasli,6.5,-
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,21900.0,0,0,1,2016-03-01,86000,Halbautomatisches Getriebe,Benzin,290.0,213.0,5417,Untersiggenthal,6.6,-
4,VW Touareg 3.0 TDI R Line Tiptronic,46900.0,0,0,1,2018-06-01,99500,Automat,Diesel,286.0,210.0,8153,Rümlang ZH,8.8,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156176,SUZUKI S-Cross 1.5 Compact+ Hybrid,34490.0,1,0,0,2025-01-01,15,Halbautomatisches Getriebe,Voll-Hybrid Benzin/Elektro,115.0,85.0,3918,Wiler (Lötschen),6.1,-
156177,SKODA Fabia 1.5 TSI Style DSG,27990.0,1,0,0,2025-01-01,25,Halbautomatisches Getriebe,Benzin,150.0,110.0,2052,Fontainemelon,-,-
156178,JAC E-JS1,17689.0,1,0,0,2025-01-01,10,-,Elektro,61.0,45.0,9427,Wolfhalden,-,330.0
156180,"JAC JAC e-S4 65,7 kWh",29989.0,1,0,0,2025-01-01,10,-,Elektro,193.0,142.0,9427,Wolfhalden,-,410.0


## Match Brand and Model

In order to make categories of the various brands and models, we imported a csv (dataframe dfb). For each car in the dataframe dfc it tries to match a brand and model from the dataframe dfb.

Some cars (aprox. 4000) couldn't be matched and were droped.

In [ ]:
# takes approx. 5 Minutes
dfc['MatchedModel'] = ''  # Initialize the column with empty strings

for index, row in dfc.iterrows():
    car = row['Car'].upper()  # Convert to uppercase to ensure case-insensitive matching
    for model in dfb['BrandModels'].str.upper():
        if model in car:
            dfc.at[index, 'MatchedModel'] = model
            break  # Stop looking for more matches once the first is found

In [ ]:
# show unmatched models, currently 4326 rows
empty_matched_model_dfc = dfc[dfc['MatchedModel'] == '']
empty_matched_model_dfc

,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,kW,ZIP,Location_Name,Verbrauch pro 100/KM (Benzin/Diesel),KM (Elektro),MatchedModel
26,RENAULT Scénic 1.3 16V Turbo Intens EDC,19900.0,0,0,1,2018-03-01,48150,Halbautomatisches Getriebe,Benzin,160.0,120.0,1024,Ecublens,5.5,-,
253,MERCEDES-BENZ GLE Coupé 63 S AMG 4matic+ *CH-F...,104900.0,0,0,1,2020-12-01,87400,Automatikgetriebe,Mild-Hybrid Benzin/Elektro,634.0,466.0,8154,Oberglatt ( Kanton ZH ),12.8,-,
274,MERCEDES-BENZ GLC Coupé 300 d AMG Line 4Matic ...,59900.0,0,0,1,2019-10-01,35900,Automat,Diesel,245.0,180.0,8620,Wetzikon,7.3,-,
286,MERCEDES-BENZ GLC Coupé 63 AMG 4Matic 9G-Tronic,69890.0,0,0,1,2019-09-01,35456,Automat,Benzin,476.0,350.0,8852,Altendorf,-,-,
324,FIAT 595C 1.4 16V TURBO ABARTH COMPETIZIONE **...,25900.0,0,0,1,2019-03-01,18900,Halbautomatisches Getriebe,Benzin,180.0,132.0,8404,Winterthur,6.7,-,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156123,MASERATI MC20 3.0 V6,279990.0,0,1,0,2025-01-01,819,Automatikgetriebe,Benzin,630.0,464.0,8404,Winterthur,11.5,-,
156145,BAW Pony,15900.0,1,0,0,2025-01-01,1,Automatikgetriebe,Elektro,30.0,22.0,2043,Boudevilliers,-,170.0,
156164,"JAC JAC e-S4 65,7 kWh",29500.0,1,0,0,2025-01-01,1,-,Elektro,193.0,142.0,1950,Sion,-,410.0,
156173,"JAC e-S4 65,7 kWh",29989.0,1,0,0,2025-01-01,10,Automatikgetriebe,Elektro,193.0,142.0,9427,Wolfhalden,-,410.0,


In [ ]:
dfc = dfc[dfc['MatchedModel'] != '']

In [ ]:
dfc

,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,kW,ZIP,Location_Name,Verbrauch pro 100/KM (Benzin/Diesel),KM (Elektro),MatchedModel
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,44450.0,0,0,1,2020-02-01,45486,Automat,Benzin,224.0,165.0,8408,Winterthur (Wülflingen),8.9,-,MERCEDES-BENZ GLB 250
1,BMW 120d xDrive Sport Line,36900.0,0,0,1,2023-05-01,28100,Automat,Diesel,190.0,140.0,7000,Chur,5.2,-,BMW 120
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,18799.0,0,0,1,2012-05-01,68200,Automat,Diesel,204.0,150.0,8156,Oberhasli,6.5,-,MERCEDES-BENZ GLK 250
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,21900.0,0,0,1,2016-03-01,86000,Halbautomatisches Getriebe,Benzin,290.0,213.0,5417,Untersiggenthal,6.6,-,SEAT LEON
4,VW Touareg 3.0 TDI R Line Tiptronic,46900.0,0,0,1,2018-06-01,99500,Automat,Diesel,286.0,210.0,8153,Rümlang ZH,8.8,-,VW TOUAREG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156175,RENAULT Mégane E-TECH EV60 Techno,44500.0,1,0,0,2025-01-01,50,-,Elektro,218.0,160.0,8259,Wagenhausen bei Stein am Rhein,-,450.0,RENAULT MÉGANE E-TECH EV60 TECHNO
156176,SUZUKI S-Cross 1.5 Compact+ Hybrid,34490.0,1,0,0,2025-01-01,15,Halbautomatisches Getriebe,Voll-Hybrid Benzin/Elektro,115.0,85.0,3918,Wiler (Lötschen),6.1,-,SUZUKI S-CROSS
156177,SKODA Fabia 1.5 TSI Style DSG,27990.0,1,0,0,2025-01-01,25,Halbautomatisches Getriebe,Benzin,150.0,110.0,2052,Fontainemelon,-,-,SKODA FABIA
156178,JAC E-JS1,17689.0,1,0,0,2025-01-01,10,-,Elektro,61.0,45.0,9427,Wolfhalden,-,330.0,JAC E-JS1


Columns renamed

In [ ]:
dfc.rename(columns={
    'Car': 'Car',
    'Price': 'Price',
    'Neuwagen': 'New Car',
    'Vorführmodell': 'Demonstration Model',
    'Occasion': 'Used Car',
    'Date': 'Date',
    'Km': 'Kilometers',
    'Transmission': 'Transmission',
    'Fuel': 'Fuel',
    'PS': 'HP',
    'kW': 'kW',
    'ZIP': 'ZIP',
    'Location_Name': 'Location Name',
    'Verbrauch pro 100/KM (Benzin/Diesel)': 'Consumption per 100 KM (Petrol/Diesel)',
    'KM (Elektro)': 'KM (Electric)',
    'MatchedModel': 'Matched Model'
}, inplace=True)

<ipython-input-35-bd1da10afbc8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc.rename(columns={


In [ ]:
dfc

,Car,Price,New Car,Demonstration Model,Used Car,Date,Kilometers,Transmission,Fuel,HP,kW,ZIP,Location Name,Consumption per 100 KM (Petrol/Diesel),KM (Electric),Matched Model
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,44450.0,0,0,1,2020-02-01,45486,Automat,Benzin,224.0,165.0,8408,Winterthur (Wülflingen),8.9,-,MERCEDES-BENZ GLB 250
1,BMW 120d xDrive Sport Line,36900.0,0,0,1,2023-05-01,28100,Automat,Diesel,190.0,140.0,7000,Chur,5.2,-,BMW 120
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,18799.0,0,0,1,2012-05-01,68200,Automat,Diesel,204.0,150.0,8156,Oberhasli,6.5,-,MERCEDES-BENZ GLK 250
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,21900.0,0,0,1,2016-03-01,86000,Halbautomatisches Getriebe,Benzin,290.0,213.0,5417,Untersiggenthal,6.6,-,SEAT LEON
4,VW Touareg 3.0 TDI R Line Tiptronic,46900.0,0,0,1,2018-06-01,99500,Automat,Diesel,286.0,210.0,8153,Rümlang ZH,8.8,-,VW TOUAREG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156175,RENAULT Mégane E-TECH EV60 Techno,44500.0,1,0,0,2025-01-01,50,-,Elektro,218.0,160.0,8259,Wagenhausen bei Stein am Rhein,-,450.0,RENAULT MÉGANE E-TECH EV60 TECHNO
156176,SUZUKI S-Cross 1.5 Compact+ Hybrid,34490.0,1,0,0,2025-01-01,15,Halbautomatisches Getriebe,Voll-Hybrid Benzin/Elektro,115.0,85.0,3918,Wiler (Lötschen),6.1,-,SUZUKI S-CROSS
156177,SKODA Fabia 1.5 TSI Style DSG,27990.0,1,0,0,2025-01-01,25,Halbautomatisches Getriebe,Benzin,150.0,110.0,2052,Fontainemelon,-,-,SKODA FABIA
156178,JAC E-JS1,17689.0,1,0,0,2025-01-01,10,-,Elektro,61.0,45.0,9427,Wolfhalden,-,330.0,JAC E-JS1


In [ ]:
new_order = ['Car',
             'Matched Model',
             'Price',
             'New Car',
             'Demonstration Model',
             'Used Car',
             'Date',
             'Kilometers',
             'Transmission',
             'Fuel',
             'Consumption per 100 KM (Petrol/Diesel)',
             'KM (Electric)',
             'HP',
             'kW',
             'ZIP',
             'Location Name']
dfc = dfc[new_order]

In [ ]:
dfc.rename(columns={ 'Demonstration Model': 'Demo Car'}, inplace=True)

## Split matched model

In order to have brand and model in two different columns, we used again the dfb dataframe. Additionaly we added the column "Additional Info" where values like "xDrive Sport Line" is stored.

In [ ]:
# compare dfc['Matched Model'] with dfb['Brands'], put every match in a new column called dfc['Brand'], put the rest of the column in dfc['Matched Model'] in the new column dfc['Type']
def find_brand_and_type(model, brands):
    for brand in brands:
        if brand in model:
            return brand, model.replace(brand, '').strip()
    return '', model.strip()

# Iterate over the DataFrame and apply the function
for index, row in dfc.iterrows():
    brand, model_type = find_brand_and_type(row['Matched Model'], dfb['Brands'])
    dfc.at[index, 'Brand'] = brand
    dfc.at[index, 'Type'] = model_type

In [ ]:
# dfc.drop(columns=['Additional Info'], inplace=True)

In [ ]:
dfc['Additional Info'] = ''

In [ ]:
dfc['Car'] = dfc['Car'].str.upper()

In [ ]:
# subtract dfc['Matched Model'] from dfc['Car'], save all results in new column dfc['Additional Info']
def subtract_model_from_car(car, matched_model):
    if pd.isnull(matched_model) or pd.isnull(car):
        return car  # Return the original car if matched_model is null
    leftover = car.replace(matched_model, '').strip()
    return leftover

# Apply the function to each row in the DataFrame and save the results in 'Additional Info'
dfc['Additional Info'] = dfc.apply(lambda row: subtract_model_from_car(row['Car'], row['Matched Model']), axis=1)

new order, drop column "Matched Model" and rename some columns

In [ ]:
new_order = ['Car',
             'Matched Model',
             'Brand',
             'Type',
             'Additional Info',
             'Price',
             'New Car',
             'Demo Car',
             'Used Car',
             'Date',
             'Kilometers',
             'Transmission',
             'Fuel',
             'Consumption per 100 KM (Petrol/Diesel)',
             'KM (Electric)',
             'HP',
             'kW',
             'ZIP',
             'Location Name']
dfc = dfc[new_order]

In [ ]:
dfc.drop(columns=['Matched Model'], inplace=True)

<ipython-input-45-00b34869e07d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc.drop(columns=['Matched Model'], inplace=True)


In [ ]:
dfc.rename(columns={
    'Price': 'Price_CHF',
    'Date': 'First_Registration',
    'Additional Info': 'Additional_Info',
    'New Car': 'New_Car',
    'Demo Car': 'Demo_Car',
    'Used Car': 'Used_Car',
    'Consumption per 100 KM (Petrol/Diesel)': 'Consumption_L_per_100_KM_Combustion',
    'KM (Electric)': 'KM_Range_Electric',
    'Location Name': 'Location_Name'
}, inplace=True)

<ipython-input-46-00fb356e9a0d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc.rename(columns={


In [ ]:
dfc.rename(columns={
    'Consumption_L_per_100_KM_Combustion': 'Consumption_Comb',
    'KM_Range_Electric': 'Range_Elec'
}, inplace=True)

<ipython-input-47-f353977cdd20>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc.rename(columns={


reset index

In [ ]:
dfc.reset_index(drop=True, inplace=True)

In [ ]:
dfc

,Car,Brand,Type,Additional_Info,Price_CHF,New_Car,Demo_Car,Used_Car,First_Registration,Kilometers,Transmission,Fuel,Consumption_Comb,Range_Elec,HP,kW,ZIP,Location_Name
0,MERCEDES-BENZ GLB 250 4MATIC AMG LINE 8G-TRONI...,MERCEDES-BENZ,GLB 250,4MATIC AMG LINE 8G-TRONIC | CH | MSI GARANTIE ...,44450.0,0,0,1,2020-02-01,45486,Automat,Benzin,8.9,-,224.0,165.0,8408,Winterthur (Wülflingen)
1,BMW 120D XDRIVE SPORT LINE,BMW,120,D XDRIVE SPORT LINE,36900.0,0,0,1,2023-05-01,28100,Automat,Diesel,5.2,-,190.0,140.0,7000,Chur
2,MERCEDES-BENZ GLK 250 CDI BLUEEFFICIENCY 4MATI...,MERCEDES-BENZ,GLK 250,CDI BLUEEFFICIENCY 4MATIC 7G-TRONIC,18799.0,0,0,1,2012-05-01,68200,Automat,Diesel,6.5,-,204.0,150.0,8156,Oberhasli
3,SEAT LEON ST 2.0 TSI CUPRA 290 DSG,SEAT,LEON,ST 2.0 TSI CUPRA 290 DSG,21900.0,0,0,1,2016-03-01,86000,Halbautomatisches Getriebe,Benzin,6.6,-,290.0,213.0,5417,Untersiggenthal
4,VW TOUAREG 3.0 TDI R LINE TIPTRONIC,VW,TOUAREG,3.0 TDI R LINE TIPTRONIC,46900.0,0,0,1,2018-06-01,99500,Automat,Diesel,8.8,-,286.0,210.0,8153,Rümlang ZH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148493,RENAULT MÉGANE E-TECH EV60 TECHNO,RENAULT,MÉGANE E-TECH EV60 TECHNO,,44500.0,1,0,0,2025-01-01,50,-,Elektro,-,450.0,218.0,160.0,8259,Wagenhausen bei Stein am Rhein
148494,SUZUKI S-CROSS 1.5 COMPACT+ HYBRID,SUZUKI,S-CROSS,1.5 COMPACT+ HYBRID,34490.0,1,0,0,2025-01-01,15,Halbautomatisches Getriebe,Voll-Hybrid Benzin/Elektro,6.1,-,115.0,85.0,3918,Wiler (Lötschen)
148495,SKODA FABIA 1.5 TSI STYLE DSG,SKODA,FABIA,1.5 TSI STYLE DSG,27990.0,1,0,0,2025-01-01,25,Halbautomatisches Getriebe,Benzin,-,-,150.0,110.0,2052,Fontainemelon
148496,JAC E-JS1,AC,J E-JS1,,17689.0,1,0,0,2025-01-01,10,-,Elektro,-,330.0,61.0,45.0,9427,Wolfhalden


# Export to csv

Data cleaning is complete!

In [ ]:
dfc.to_csv('Final_AutoScout.csv')